In [153]:
from numpy import *
import json
import datetime
import numpy as np  

linlist = []

In [154]:
#讀檔2 目前沒用到
def readDataFile(file_name):
    dataFile = open(file_name)          #開檔
    lines = dataFile.read().split('\n') #每行分開，存到lines， lines下面會用到
    dataFile.close()                    #讀完關掉
    
    for i in range(len(lines)):             #lines有幾行就做幾次
        record = lines[i].split(' ')        #用' '分開空格
        print record[:-1]                   #print出來看看
        for j in range(len(record)-1):      #最後會有一個莫名其妙的空值 故用len(record)-1修正矩陣長度-1
            coord[i][j]=coord[i][record[j]] #匯入coord 我們最後回傳的矩陣就是coord
    return coord                            #回傳coord

In [155]:
#讀檔
def readDataFile(file_name,linlist):
    dataFile = open(file_name)          #開檔
    lines = dataFile.read().split('\n') #每行分開，存到lines， lines下面會用到
    dataFile.close()                    #讀完關掉
    
    for i in range(len(lines)-1):                   #lines有幾行就做幾次
        linlist.append(map(int, lines[i].split()))  #用map會直接轉成int格式! 並且分開每個數值
    return linlist                                  #回傳linlist 

In [156]:
gj=readDataFile('input.txt',linlist)     #將此二維陣列存到gj 等等直接呼叫

In [157]:
gj    #看看有沒有錯

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29],
 [30, 31, 32],
 [33, 34, 35],
 [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46],
 [38, 39, 47, 48],
 [38, 39, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58],
 [32, 41, 59, 60, 61, 62],
 [3, 39, 48],
 [63, 64, 65, 66, 67, 68],
 [32, 69],
 [48, 70, 71, 72],
 [39, 73, 74, 75, 76, 77, 78, 79],
 [36, 38, 39, 41, 48, 79, 80, 81],
 [82, 83, 84],
 [41, 85, 86, 87, 88],
 [39, 48, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101],
 [36, 38, 39, 48, 89],
 [39, 41, 102, 103, 104, 105, 106, 107, 108],
 [38, 39, 41, 109, 110],
 [39, 111, 112, 113, 114, 115, 116, 117, 118],
 [119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133],
 [48, 134, 135, 136],
 [39, 48, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149],
 [39, 150, 151, 152],
 [38, 39, 56, 153, 154, 155],
 [48, 156, 157, 158, 159, 160],
 [39, 

In [158]:
def createC1(dataSet):
    C1 = [] #C1是大小為1的所有候選集的集合
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])             
    C1.sort()
    return map(frozenset, C1)#use frozen set so we can use it as a key in a dict    

In [159]:
#D表示DataSet，CK表示候選集，minSupport表示最小的支持度，自己設定
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not ssCnt.has_key(can): ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = [] #儲存满足最小支持度要求的項集
    supportData = {} #每個項集的支持度字典
    for key in ssCnt:  #計算所有項集的支持度
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [160]:
#LK是頻繁項集列表，K表示接下來合併的項集中的單個想的個數 {1,2,3} 表示k=3
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2] #前k-2個項次相同時，將兩個集合合併
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList 

In [161]:
#只需要输入数据集和支持度即可
def apriori(dataSet, minSupport = 0.3):
    C1 = createC1(dataSet)
    D = map(set, dataSet)
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData#返回頻繁項集和每個項集的支持度值

In [162]:
#輸入的參數分別為：頻繁項、支持度data字典、自定義的最小支持度，返回的是可信度規則列表
def generateRules(L, supportData, minConf=0.1):  #支持度是通過scanD得到的字典
    bigRuleList = []
    for i in range(1, len(L)):#只去頻繁項集中元素個数大於2的子集，如{1,2}{1,2,3}，不取{2}{3},etc...
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

In [163]:
#計算可信度，找到滿足最小可信度的要求規則
def calcConf(freqSet, H, supportData, brl, minConf=0.2):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print freqSet-conseq,'-->',conseq,'conf:',conf
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [164]:
#從最初的項集中生成更多的規則
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.2):   #預設minConf=0.2 可由使用者呼叫時直接調整
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #進一步合併項集
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [166]:
rule

[(frozenset({78}), frozenset({39}), 0.7446808510638298),
 (frozenset({3981}), frozenset({39}), 0.810810810810811),
 (frozenset({2958}), frozenset({48}), 0.8043478260869564),
 (frozenset({334}), frozenset({39}), 0.7916666666666667),
 (frozenset({110}), frozenset({38}), 0.9685863874345548),
 (frozenset({281}), frozenset({38}), 0.9677419354838711),
 (frozenset({37}), frozenset({38}), 1.0),
 (frozenset({255}), frozenset({48}), 0.7592592592592593),
 (frozenset({105}), frozenset({38}), 0.9268292682926829),
 (frozenset({1146}), frozenset({39}), 0.7076923076923077),
 (frozenset({49}), frozenset({48}), 0.7681159420289855),
 (frozenset({79}), frozenset({39}), 0.7096774193548387),
 (frozenset({170}), frozenset({38}), 0.9791666666666666),
 (frozenset({47}), frozenset({38}), 0.9705882352941176),
 (frozenset({36}), frozenset({38}), 0.9427083333333334),
 (frozenset({840}), frozenset({38}), 0.96875),
 (frozenset({55}), frozenset({38}), 0.868421052631579),
 (frozenset({175}), frozenset({39}), 0.7045454

In [167]:
#正式的
start = datetime.datetime.now() #紀錄時間
 
L,supportData = apriori(gj,0.1)           #執行apriori  minimum support =0.1   輸出存到 L supportData
rule = generateRules(L,supportData,0.2)   #generateRules  一樣 存到rule

end = datetime.datetime.now()   #程式運行結束時間
runtime = end - start           #計算runtime=end-start

frozenset([39]) --> frozenset([41]) conf: 0.225239269857
frozenset([41]) --> frozenset([39]) conf: 0.763733690197
frozenset([39]) --> frozenset([48]) conf: 0.575076467686
frozenset([48]) --> frozenset([39]) conf: 0.691634033464
frozenset([39]) --> frozenset([38]) conf: 0.204144055254
frozenset([38]) --> frozenset([39]) conf: 0.663311105412
frozenset([41]) --> frozenset([48]) conf: 0.603412512546
frozenset([48]) --> frozenset([41]) conf: 0.214026343895


In [168]:
runtime

datetime.timedelta(0, 168, 196000)

In [169]:
def store(data,path):
    with open(path,'w') as f:
        f.write(data)

In [170]:
freItem_list = []
for tmp in L:
    if len(tmp) !=0:
        for item in tmp:
            freItem_list.append(list(item))
freItem_json = json.dumps(freItem_list,indent=4)
store(freItem_json,'/Users/user/Desktop/freItem.json')

In [171]:
freItem_list

[[41], [39], [32], [38], [48], [41, 39], [48, 39], [38, 39], [48, 41]]

In [172]:
supportData_list = []
for key, value in supportData.iteritems():
    s = str(list(key))+':'+str(value)
    supportData_list.append(s)
supportData_list.sort()
supportData_json = json.dumps(supportData_list,indent=4)
store(supportData_json,'/Users/user/Desktop/supportData.json')

In [173]:
strongRule = []
for tmp in rule:
    s = str(list(tmp[0])[0])+'-->'+str(list(tmp[1])[0])+'---'+str(tmp[2])
    strongRule.append(s)
strongRule_json = json.dumps(strongRule,indent=4)
store(strongRule_json,'/Users/user/Desktop/strongRule.json')

In [174]:
strongRule

['39-->41---0.225239269857',
 '41-->39---0.763733690197',
 '39-->48---0.575076467686',
 '48-->39---0.691634033464',
 '39-->38---0.204144055254',
 '38-->39---0.663311105412',
 '41-->48---0.603412512546',
 '48-->41---0.214026343895']

In [175]:
linlist

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29],
 [30, 31, 32],
 [33, 34, 35],
 [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46],
 [38, 39, 47, 48],
 [38, 39, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58],
 [32, 41, 59, 60, 61, 62],
 [3, 39, 48],
 [63, 64, 65, 66, 67, 68],
 [32, 69],
 [48, 70, 71, 72],
 [39, 73, 74, 75, 76, 77, 78, 79],
 [36, 38, 39, 41, 48, 79, 80, 81],
 [82, 83, 84],
 [41, 85, 86, 87, 88],
 [39, 48, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101],
 [36, 38, 39, 48, 89],
 [39, 41, 102, 103, 104, 105, 106, 107, 108],
 [38, 39, 41, 109, 110],
 [39, 111, 112, 113, 114, 115, 116, 117, 118],
 [119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133],
 [48, 134, 135, 136],
 [39, 48, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149],
 [39, 150, 151, 152],
 [38, 39, 56, 153, 154, 155],
 [48, 156, 157, 158, 159, 160],
 [39, 

In [148]:
#另一組測試資料點
dataSet = loadDataSet()             #LoadData
L,suppoData = apriori(gj,0.005)      #計算頻繁項
rules = generateRules(L,suppoData,minConf=0.5) #抽取規則

frozenset([78]) --> frozenset([39]) conf: 0.744680851064
frozenset([246]) --> frozenset([48]) conf: 0.650793650794
frozenset([3981]) --> frozenset([39]) conf: 0.810810810811
frozenset([9]) --> frozenset([48]) conf: 0.510638297872
frozenset([749]) --> frozenset([48]) conf: 0.509933774834
frozenset([2958]) --> frozenset([48]) conf: 0.804347826087
frozenset([189]) --> frozenset([48]) conf: 0.525252525253
frozenset([179]) --> frozenset([48]) conf: 0.576923076923
frozenset([2413]) --> frozenset([39]) conf: 0.682926829268
frozenset([36]) --> frozenset([39]) conf: 0.682291666667
frozenset([66]) --> frozenset([39]) conf: 0.578947368421
frozenset([592]) --> frozenset([39]) conf: 0.630952380952
frozenset([1121]) --> frozenset([48]) conf: 0.55900621118
frozenset([334]) --> frozenset([39]) conf: 0.791666666667
frozenset([208]) --> frozenset([48]) conf: 0.620689655172
frozenset([110]) --> frozenset([38]) conf: 0.968586387435
frozenset([812]) --> frozenset([48]) conf: 0.547169811321
frozenset([783])

In [149]:
rules = generateRules(L,suppoData,minConf=0.5)

frozenset([78]) --> frozenset([39]) conf: 0.744680851064
frozenset([246]) --> frozenset([48]) conf: 0.650793650794
frozenset([3981]) --> frozenset([39]) conf: 0.810810810811
frozenset([9]) --> frozenset([48]) conf: 0.510638297872
frozenset([749]) --> frozenset([48]) conf: 0.509933774834
frozenset([2958]) --> frozenset([48]) conf: 0.804347826087
frozenset([189]) --> frozenset([48]) conf: 0.525252525253
frozenset([179]) --> frozenset([48]) conf: 0.576923076923
frozenset([2413]) --> frozenset([39]) conf: 0.682926829268
frozenset([36]) --> frozenset([39]) conf: 0.682291666667
frozenset([66]) --> frozenset([39]) conf: 0.578947368421
frozenset([592]) --> frozenset([39]) conf: 0.630952380952
frozenset([1121]) --> frozenset([48]) conf: 0.55900621118
frozenset([334]) --> frozenset([39]) conf: 0.791666666667
frozenset([208]) --> frozenset([48]) conf: 0.620689655172
frozenset([110]) --> frozenset([38]) conf: 0.968586387435
frozenset([812]) --> frozenset([48]) conf: 0.547169811321
frozenset([783])

In [150]:
L

[[frozenset({3735}),
  frozenset({1239}),
  frozenset({479}),
  frozenset({310}),
  frozenset({1867}),
  frozenset({727}),
  frozenset({1020}),
  frozenset({1198}),
  frozenset({1513}),
  frozenset({785}),
  frozenset({438}),
  frozenset({4251}),
  frozenset({1327}),
  frozenset({2552}),
  frozenset({675}),
  frozenset({3641}),
  frozenset({1417}),
  frozenset({846}),
  frozenset({396}),
  frozenset({1600}),
  frozenset({856}),
  frozenset({150}),
  frozenset({147}),
  frozenset({338}),
  frozenset({806}),
  frozenset({1257}),
  frozenset({1783}),
  frozenset({1956}),
  frozenset({371}),
  frozenset({270}),
  frozenset({66}),
  frozenset({592}),
  frozenset({45}),
  frozenset({209}),
  frozenset({571}),
  frozenset({260}),
  frozenset({426}),
  frozenset({110}),
  frozenset({18}),
  frozenset({1872}),
  frozenset({861}),
  frozenset({488}),
  frozenset({2714}),
  frozenset({161}),
  frozenset({855}),
  frozenset({408}),
  frozenset({1135}),
  frozenset({155}),
  frozenset({1126}),
  fr

In [151]:
rules

[(frozenset({78}), frozenset({39}), 0.7446808510638298),
 (frozenset({246}), frozenset({48}), 0.6507936507936508),
 (frozenset({3981}), frozenset({39}), 0.810810810810811),
 (frozenset({9}), frozenset({48}), 0.5106382978723404),
 (frozenset({749}), frozenset({48}), 0.509933774834437),
 (frozenset({2958}), frozenset({48}), 0.8043478260869564),
 (frozenset({189}), frozenset({48}), 0.5252525252525252),
 (frozenset({179}), frozenset({48}), 0.576923076923077),
 (frozenset({2413}), frozenset({39}), 0.6829268292682927),
 (frozenset({36}), frozenset({39}), 0.6822916666666667),
 (frozenset({66}), frozenset({39}), 0.5789473684210527),
 (frozenset({592}), frozenset({39}), 0.6309523809523809),
 (frozenset({1121}), frozenset({48}), 0.5590062111801243),
 (frozenset({334}), frozenset({39}), 0.7916666666666667),
 (frozenset({208}), frozenset({48}), 0.6206896551724138),
 (frozenset({110}), frozenset({38}), 0.9685863874345548),
 (frozenset({812}), frozenset({48}), 0.5471698113207547),
 (frozenset({783})

In [152]:
suppoData

{frozenset({2212, 2344}): 0.0001818512456810329,
 frozenset({264, 381}): 0.0001818512456810329,
 frozenset({2609}): 0.002182214948172395,
 frozenset({2379}): 0.0014548099654482633,
 frozenset({548, 2329}): 0.0001818512456810329,
 frozenset({78, 441}): 0.0003637024913620658,
 frozenset({319}): 0.0030914711765775596,
 frozenset({170, 179}): 0.0003637024913620658,
 frozenset({66, 237}): 0.0001818512456810329,
 frozenset({301, 3693}): 0.0003637024913620658,
 frozenset({3209}): 0.0005455537370430987,
 frozenset({38, 41, 677}): 0.0010911074740861974,
 frozenset({2694, 3966}): 0.0001818512456810329,
 frozenset({2671}): 0.0003637024913620658,
 frozenset({796}): 0.0018185124568103291,
 frozenset({365, 504}): 0.0001818512456810329,
 frozenset({179, 704}): 0.0001818512456810329,
 frozenset({60, 1568}): 0.0005455537370430987,
 frozenset({66, 1344}): 0.0001818512456810329,
 frozenset({816}): 0.0014548099654482633,
 frozenset({340, 865}): 0.0001818512456810329,
 frozenset({334, 1198}): 0.00054555373